# ModOAP - Calculs lexicométriques sur un corpus

Ce script permet de calculer des informations lexicométriques d'une liste de termes au sein d'un corpus de textes numérisés.

La liste de termes à rechercher doit être fournie dans la dernière cellule sous forme d'un fichier texte contenant un terme par ligne.

Le corpus à interroger doit être fourni sous forme d'un dossier contenant un ou plusieurs documents Gallica au format json (issus d'un téléchargement depuis Gallica grâce au script Modoap - Téléchargement de texte structuré) (un fichier par document).

Lors d'une première utilisation, le corpus doit subir un pré-traitement linguistique (création d'un fichier .nlp par fichier .json).

Une fois les fichiers .nlp générés, le corpus pré-traité peut être rapidement importé.

Le résultat des calculs est un fichier tableau Google Sheet créé et partagé sur le compte synchronisé, contenant les informations. 

Ces résultats peuvent ensuite être représentés dans une interface Web (voir :https://modoap.huma-num.fr/romans_scolaires/)





**Calculs lexicométriques réalisés :**

- Nombre total de mots du corpus
- Compte des occurrences de chaque terme
- Répartition des termes dans le corpus
- Répartition des termes par :
  - titre de document
  - année de publication
  - éditeur de document
  - auteur de document
- Fréquences des co-occurrences du terme dans une fenêtre donnée :
  - pour toutes les co-occurrences
  - pour les noms uniquement
  - pour les verbes uniquement
  - pour les adjectifs uniquement

- Concordances des occurrences de chaque terme dans une fenêtre donnée


In [ ]:
#@title ##  Préparation

#@markdown ### Lancer cette cellule en préambule
#@markdown Puis cliquer sur le lien généré pour synchroniser un compte Google Drive si demandé

# chargement d'un google drive
import os
from google.colab import drive

if not os.path.exists("/content/drive/MyDrive/") :
  drive.mount('/content/drive/')

import pickle
!pip install -q -U spacy
import spacy
!python -m spacy download fr_core_news_lg
from spacy.tokens import DocBin

import fr_core_news_lg

nlp = fr_core_news_lg.load() ## PIPE SPEC
nlp = spacy.load("fr_core_news_lg", disable=["attribute_ruler", "textcat_multilabel", "textcat", "ner", "entity_linker", "entity_ruler", "entity_linker"])
nlp.max_length = 1500000

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.probability import FreqDist

import matplotlib.pyplot as plt

import numpy as np
import json
import glob
import pandas as pd
import altair as alt
from functools import partial
from tqdm import tqdm
tqdm = partial(tqdm, position=0, leave=True)

!pip install colorama

from colorama import Fore, Back, Style
import re


def lemmatisation(doc):
  lemmes = [mot.lemma_ for mot in doc if mot.pos_ is not "PUNCT" and mot.pos_ is not "SPACE"]
  return lemmes
  
def stopwords_filter(doc):
  mots_grammaticaux = set(stopwords.words('french'))
  add = ["aussi", "quelque", "quelques", "tel", "telle", "telles", "tels", "y"]
  for mot in add :
    mots_grammaticaux.add(mot)
  POS_filter = ["SPACE", "ADP", "DET", "CCONJ", "PUNCT", "NUM", "PRON"]
  chars = list(set('$,./?\;\'\'\’\’\\^')) + ["\n","\xa0"]
  mots_lexicaux = [mot for mot in doc if len(mot.text) > 1 and mot.text.lower() not in mots_grammaticaux and not any(pos in mot.pos_ for pos in POS_filter) and not any((c in chars) for c in mot.text)]
  return mots_lexicaux

def lemma_stopwords_filter(doc):
  mots_grammaticaux = set(stopwords.words('french'))
  add = ["aussi", "quelque", "quelques", "tel", "telle", "telles", "tels"]
  for mot in add :
    mots_grammaticaux.add(mot)
  POS_filter = ["SPACE", "ADP", "DET", "CCONJ", "PUNCT", "NUM", "PRON"]
  chars = list(set('$,./?\;\'\'\’\’\\^')) + ["\n","\xa0"]
  mots_lexicaux = [mot.lemma_ for mot in doc if len(mot.text) > 1 and mot.text.lower() not in mots_grammaticaux and not any(pos in mot.pos_ for pos in POS_filter) and not any((c in chars) for c in mot.text)]
  return mots_lexicaux 

def premier_filtre(doc):
  POS_filter = ["SPACE", "PUNCT"]
  chars = list(set('$,./?\;\'\'\’\’\\^')) + ["\n","\xa0"]
  mots_filtres = [mot for mot in doc if len(mot.text) > 1 and not any((c in chars) for c in mot.text) and not any(pos in mot.pos_ for pos in POS_filter)]
  return mots_filtres

def json2dict(jsonfile) :
  with open(jsonfile, "r") as json_in :
    dictjson = json.load(json_in)
  return dictjson

def nettoyage_bloc(bloc):
  bloc = bloc.replace(" fig ", "")
  bloc = bloc.replace(" do ", " de ")
  bloc = bloc.replace(" dos ", " des ")
  bloc = bloc.replace(" * ", "")
  bloc = bloc.replace(" k ", "")
  bloc = bloc.replace("«", "")
  bloc = bloc.replace("»", "")
  bloc = bloc.replace(" lo ", " le ")
  bloc = bloc.replace(" los ", " les ")
  bloc = bloc.replace(" ot ", " et ")
  bloc = bloc.replace(" ôt ", " et ")
  bloc = bloc.replace("- ", "")
  bloc = bloc.replace(" - ", "")
  bloc = bloc.replace("  ", " ")
  bloc = bloc.replace("   ", " ")
  bloc = bloc.replace(" so ", " se ")
  bloc = bloc.replace(" quo ", " que ")
  bloc = bloc.replace(" Ello ", " Elle ")
  bloc = bloc.replace(" ello ", " elle ")
  bloc = bloc.replace(" villo ", " ville ")
  bloc = bloc.replace(" ollo ", " elle ")
  bloc = bloc.replace(" uno ", " une ")
  bloc = bloc.replace(" Cotte ", " Cette ")
  return bloc


def pretraitements_paragraphe(dictjson) :
  docs = []
  for tb in dictjson["Text_Blocks"]:
    bloc = nettoyage_bloc(dictjson["Text_Blocks"][tb]["Content"])
    docs.append(nlp(bloc))
  doc_bin = DocBin(docs=docs)
  return doc_bin

def pre_traitement_dossier(chemin_dossier):
  doc_bins = []
  nlp_traites = [nlpfile for nlpfile in glob.glob(os.path.join(chemin_dossier,"*.nlp"))]
  for jsonfile in tqdm(glob.glob(os.path.join(chemin_dossier,"*.json"))) :
    if jsonfile[:-5]+".nlp" not in nlp_traites :
      dictjson = json2dict(jsonfile)
      doc_bin = pretraitements_paragraphe(dictjson)
      doc_bin.to_disk(jsonfile[:-5]+".nlp")
      doc_bins.append(pretraitements_paragraphe(dictjson))
  return doc_bins

# Définition du corpus




In [ ]:
#@markdown # Prétraiter un dossier corpus

#@markdown #### Entrer le chemin d'un dossier contenant des documents au format .json à pré-traiter :
chemin_dossier = "" #@param {type:"string"}
#@markdown Crée un fichier .nlp par fichier json dans le même dossier

#@markdown Exemple de chemin: /content/drive/My Drive/datasets/

#@markdown Le temps de calcul est long, nécessite d'être lancé une fois pour toutes.

doc_bins_corpus = pre_traitement_dossier(chemin_dossier)

In [ ]:
#@markdown # Importer un dossier corpus prétraité:

#@markdown #### Entrer le chemin d'un dossier contenant des documents déjà pré-traités, au format .nlp :
chemin_dossier = "" #@param {type:"string"}

#@markdown Exemple de chemin: /content/drive/My Drive/datasets/

dico_corpus = {}

jsons = [fil for fil in glob.glob(os.path.join(chemin_dossier,"*.json"))]
nlpz = [fil for fil in glob.glob(os.path.join(chemin_dossier,"*.nlp"))]

for json_file in tqdm(jsons) :
  nlpfile = json_file[:-5]+".nlp"
  dict_man = json2dict(json_file)
  doc_bin = DocBin().from_disk(nlpfile)
  docs = list(doc_bin.get_docs(nlp.vocab))
  dict_man['Infos_Doc']['docs'] = docs
  dict_man['Infos_Doc']['nlp'] = nlpfile
  i = 0
  for tb in dict_man['Text_Blocks'] :
    dict_man['Text_Blocks'][tb]["Num_Paragraphe"] = i
    i+= 1
  dico_corpus[json_file] = dict_man

# Transformer un JSON_CORPUS pour que l'ID des tb soit [1,2,3..]
dico_corpus_ids = {}
for man in tqdm(dico_corpus) :
  i = 0
  infodoc = dico_corpus[man]["Infos_Doc"]
  tbz = {}
  for tb in dico_corpus[man]["Text_Blocks"] :
    tbz[i] = dico_corpus[man]["Text_Blocks"][tb]
    i+=1
  manew = {"Infos_Doc" : infodoc, "Text_Blocks" : tbz}
  dico_corpus_ids[man] = manew

dico_corpus = dico_corpus_ids

print()
print("{} documents importés".format(len(jsons)))
print(nlpz)

# Calculs lexicométriques

In [ ]:
### CALCUL DES PERSONNAGES OK +++++++++++++++ AJOUT DES FREQS RELATIVES


from spacy.matcher import Matcher
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())


#@markdown ## Calculs Lexicométriques
#@markdown Renseigner les paramètres avant de lancer la cellule

#@markdown ---
#@markdown #### Entrer le chemin vers un fichier texte contenant les termes à repérer :
chemin_fichier_termes = "/content/drive/MyDrive/100PERSOS.txt" #@param {type:"string"}
#@markdown Un fichier .txt contenant les termes à rechercher dans le corpus : un terme par ligne. 

#@markdown ---
#@markdown #### Entrer le nom du tableau Google Sheet à créer :
nom_tableau = "Personnages" #@param {type:"string"}
#@markdown Un fichier Google Sheet portant ce nom sera créé sur le Drive

#@markdown ---
#@markdown #### Définir la fenêtre de voisinage pour le calcul des co-occurrences et des concordances :
fenetre_voisinage = 30 #@param {type:"integer"}
#@markdown Correspond au nombre de mots dans les voisinages gauche et droit de chaque occurrence considérée. 
#@markdown Par défaut : 30 

#@markdown ---
#@markdown #### Ajouter des termes à filtrer lors du calcul, séparés par / :
mots_a_filtrer = "" #@param {type:"string"}
#@markdown Exemple : que/faire/il filtrera les termes "que", "faire" et "il" qui ne seront pas pris en compte et n'apparaîtront pas dans les résultats.


#@markdown ---


if not fenetre_voisinage :
  fenetre_voisinage = 30

sh = gc.create(nom_tableau) 

liste_stop = ["-", "y", "1", ",",",", "er",".",",", ""]
liste_stop = liste_stop + [x for x in mots_a_filtrer.split("/")]

with open(chemin_fichier_termes, "r") as listin : 
  listpers = listin.read()

liste_persos = [x.strip() for x in listpers.split("\n")]
freqTotales_personnages = {}
dict_nbmots_romans = {}

print()
print("Calculs lexicométriques sur {} termes et création d'un fichier Google Sheet contenant les résultats : ".format(len(liste_persos)))

for perso in tqdm(liste_persos) :  


  matcher = Matcher(nlp.vocab)

  if "Darc" in perso :
    pattern = [{"LOWER": "jeanne"},{"LOWER": "darc"} ] 
    pattern2 = [{"LOWER": "jeanne"}, {"LENGTH": 2}, {"LOWER": "arc"}] 
    matcher.add("custom", [pattern, pattern2])

  elif "Napoléon_Ier" in perso :
    pattern = [{"LOWER": {"NOT_IN" : ["louis-"]}}, {"LOWER": "napoléon"},{"LOWER": {"NOT_IN" : ["iii","111"]}}]  
    matcher.add("custom", [pattern])

  elif "Napoléon_III" in perso :
    pattern = [{"LOWER": "napoléon"},{"LOWER": "iii"} ] 
    pattern2 = [{"LOWER": "louis-"}, {"LOWER": "napoléon"}]
    pattern3 = [{"LOWER": "napoléon"}, {"ORTH": "111"}] 
    matcher.add("custom", [pattern, pattern2, pattern3])

  elif"Quint" in perso :
    pattern = [{"LEMMA": "Charles"}, {"TEXT": "-"} ,{"LEMMA": "Quint"}]
    matcher.add("custom", [pattern])

  elif"Duguay" in perso :
    pattern = [{"LEMMA": "Duguay"}, {"TEXT": "-"} ,{"LEMMA": "Trouin"}]
    matcher.add("custom", [pattern])

  elif " " in perso :
    tokenlist = perso.split(" ")
    if len(tokenlist) == 2 :
      pattern = [{"TEXT": tokenlist[0] },{"TEXT": tokenlist[1]} ] 
      matcher.add("custom", [pattern])

    elif len(tokenlist) == 3 :
      pattern = [{"TEXT": tokenlist[0] },{"TEXT": tokenlist[1]},{"TEXT": tokenlist[2]} ] 
      matcher.add("custom", [pattern])

    else : print("Problème avec le nom du personnage")

  else :
    pattern = [{"TEXT": perso}] 
    matcher.add("custom", [pattern])

  cooc_global = []
  cooc_adj = []
  cooc_verb = []
  cooc_nom = []
  resultats_annees = {}
  resultats_romans = {}
  resultats_editeur = {}
  resultats_auteur = {}
  concordances = [] 

  # Itération sur les romans du corpus
  patrons = []
  compteur_matches = 0
  for man in dico_corpus :

    nb_mots_roman = 0
    annee = dico_corpus[man]["Infos_Doc"]["Publication_Date"]
    roman = dico_corpus[man]["Infos_Doc"]["Titre"]
    editeur = dico_corpus[man]["Infos_Doc"]["Editeur"]
    auteur = dico_corpus[man]["Infos_Doc"]["Auteur"]
    
    docs = dico_corpus[man]["Infos_Doc"]["docs"]
    compteur_paragraphes = 0

    # Itération sur les blocs texte d'un roman
    for doc in docs :
      nb_mots_paragraphe = len([w.text for w in doc if w.pos_ != "PUNCT"])
      nb_mots_roman = nb_mots_roman + nb_mots_paragraphe
    
      matches = matcher(doc)
      
      # Itération sur les patrons repérés d'un bloc texte
      for match_id, start, end in matches:
        string_id = nlp.vocab.strings[match_id]  
        span = doc[start:end]
        patrons.append(span.text)

        # Score Freqs totales
        if perso in freqTotales_personnages : 
          freqTotales_personnages[perso] += 1
        else :
          freqTotales_personnages[perso] = 1

      # Récupération de la concordance
        #Contenu = dico_corpus[man]["Text_Blocks"][compteur_paragraphes]["Content"]
        Roman	= roman
        Page_Num = dico_corpus[man]["Text_Blocks"][compteur_paragraphes]["Page_Num"]
        if "Napoléon_Ier" in perso :
          Concordance = str(doc[start-(fenetre_voisinage+1) : start+1]) + " <mark>"+str(doc[start+1:end-1])+"</mark> "+ str(doc[end-1:end+1+fenetre_voisinage])
        else :  
          Concordance = str(doc[start-fenetre_voisinage : start]) + " <mark>"+str(span)+"</mark> "+ str(doc[end:end+fenetre_voisinage])
        Region = dico_corpus[man]["Text_Blocks"][compteur_paragraphes]["Position"]
        Url_iiif_region = "https://gallica.bnf.fr/iiif/ark:/12148/{}/f{}/{},{},{},{}/full/0/native.jpg".format(man.split("_")[-3], Page_Num, Region["hpos"], Region["vpos"], Region["width"], Region["height"])
        Lien_iiif_region = "<a href=\""+Url_iiif_region+"\">Voir le paragraphe</a>"
        Url_gallica_Page = "https://gallica.bnf.fr/ark:/12148/{}/f{}.item.texteImage".format(man.split("_")[-3], Page_Num)
        Lien_gallica_Page = "<a href=\""+Url_gallica_Page+"\">Voir la page</a>"

        dict_concord = {"Roman" : roman,
                        "Page_Num": Page_Num,
                        "Concordance" : Concordance,
                        "Url_iiif_region" : Lien_iiif_region,
                        "Url_gallica_Page" : Lien_gallica_Page}
        concordances.append(dict_concord)

      # Comptes par année auteur éditeur et roman :
        compteur_matches+=1
        if annee in resultats_annees :
          resultats_annees[annee] += 1
        else :
          resultats_annees[annee] = 1

        if roman in resultats_romans :
          resultats_romans[roman] += 1
        else :
          resultats_romans[roman] = 1
          
        if auteur in resultats_auteur :
          resultats_auteur[auteur] += 1
        else :  
          resultats_auteur[auteur] = 1
        if editeur in resultats_editeur :
          resultats_editeur[editeur] += 1
        else : 
          resultats_editeur[editeur] = 1

      # Comptes des Cooccurrences
        if cooc_global : 
          cooc_global = cooc_global + [str(w.lemma_) for w in doc[start-fenetre_voisinage:start]if w.is_stop == False and w.pos_ is not "PUNCT" and w.lemma_ not in liste_stop] + [str(w.lemma_) for w in doc[end:end+fenetre_voisinage]if w.is_stop == False and w.pos_ is not "PUNCT" and w.lemma_ not in liste_stop]
        else :
          cooc_global = [str(w.lemma_) for w in doc[start-fenetre_voisinage:start]if w.is_stop == False and w.pos_ is not "PUNCT" and w.lemma_ not in liste_stop] + [str(w.lemma_) for w in doc[end:end+fenetre_voisinage]if w.is_stop == False and w.pos_ is not "PUNCT" and w.lemma_ not in liste_stop]

        if cooc_adj :
          cooc_adj = cooc_adj + [str(w.lemma_) for w in doc[start-fenetre_voisinage:start]if w.is_stop == False and w.pos_ == "ADJ" and w.pos_ is not "PUNCT" and w.lemma_ not in liste_stop] + [str(w.lemma_) for w in doc[end:end+fenetre_voisinage]if w.is_stop == False and w.pos_ == "ADJ" and w.pos_ is not "PUNCT" and w.lemma_ not in liste_stop]
        else :
          cooc_adj = [str(w.lemma_) for w in doc[start-fenetre_voisinage:start]if w.is_stop == False and w.pos_ == "ADJ" and w.pos_ is not "PUNCT" and w.lemma_ not in liste_stop] + [str(w.lemma_) for w in doc[end:end+fenetre_voisinage]if w.is_stop == False and w.pos_ == "ADJ" and w.pos_ is not "PUNCT" and w.lemma_ not in liste_stop]

        if cooc_verb :
          cooc_verb = cooc_verb + [str(w.lemma_) for w in doc[start-fenetre_voisinage:start]if w.is_stop == False and w.pos_ == "VERB" and w.pos_ is not "PUNCT" and w.lemma_ not in liste_stop] + [str(w.lemma_) for w in doc[end:end+fenetre_voisinage]if w.is_stop == False and w.pos_ == "VERB" and w.pos_ is not "PUNCT" and w.lemma_ not in liste_stop]
        else :
          cooc_verb = [str(w.lemma_) for w in doc[start-fenetre_voisinage:start]if w.is_stop == False and w.pos_ == "VERB" and w.pos_ is not "PUNCT" and w.lemma_ not in liste_stop] + [str(w.lemma_) for w in doc[end:end+fenetre_voisinage]if w.is_stop == False and w.pos_ == "VERB" and w.pos_ is not "PUNCT" and w.lemma_ not in liste_stop]

        if cooc_nom :
          cooc_nom = cooc_nom + [str(w.lemma_) for w in doc[start-fenetre_voisinage:start]if w.is_stop == False and w.pos_ == "NOUN" and w.pos_ is not "PUNCT" and w.lemma_ not in liste_stop] + [str(w.lemma_) for w in doc[end:end+fenetre_voisinage]if w.is_stop == False and w.pos_ == "NOUN" and w.pos_ is not "PUNCT" and w.lemma_ not in liste_stop]
        else :
          cooc_nom = [str(w.lemma_) for w in doc[start-fenetre_voisinage:start]if w.is_stop == False and w.pos_ == "NOUN" and w.pos_ is not "PUNCT" and w.lemma_ not in liste_stop] + [str(w.lemma_) for w in doc[end:end+fenetre_voisinage]if w.is_stop == False and w.pos_ == "NOUN" and w.pos_ is not "PUNCT" and w.lemma_ not in liste_stop]

      compteur_paragraphes += 1
    if roman not in dict_nbmots_romans.keys():
      dict_nbmots_romans[roman] = nb_mots_roman
  nb_mots_corpus = sum(dict_nbmots_romans.values())
  # Création du tableau de concordances :

  dataviz_conco = [[k for k, v in concordances[0].items()]]
  for dict_concord in concordances :
    row = [v for k, v in dict_concord.items()]
    dataviz_conco.append(row)

  # Création des tableaux de fréquences par année, auteur, editeur et roman :

  dataviz_freq_annees = [["annee", "frequence"]]
  for annee, freq in resultats_annees.items() :
    liste = [annee, freq]
    dataviz_freq_annees.append(liste)

  dataviz_freq_roman = [["roman", "frequence"]]
  for roman, freq in resultats_romans.items() :
    liste = [roman, freq]
    dataviz_freq_roman.append(liste)

  dataviz_freq_editeur = [["editeur", "frequence"]]
  for editeur, freq in resultats_editeur.items() :
    liste = [editeur, freq]
    dataviz_freq_editeur.append(liste)

  dataviz_freq_auteur = [["auteur", "frequence"]]
  for auteur, freq in resultats_auteur.items() :
    liste = [auteur, freq]
    dataviz_freq_auteur.append(liste)

  # Création des tableaux des fréquences des cooccurrences 

  fdistcooc_global = FreqDist(cooc_global)
  fdistcooc_globalMC = fdistcooc_global.most_common(10)
  dataviz_cooc_global = [["terme", "freq_absolue","freq_relative"]]
  for e in fdistcooc_globalMC :
    liste = [e[0],e[1],e[1]/nb_mots_corpus*100]
    dataviz_cooc_global.append(liste)


  #################################

  fdistcooc_adj = FreqDist(cooc_adj)
  fdistcooc_adjMC = fdistcooc_adj.most_common(10)
  dataviz_cooc_adj = [["terme", "freq_absolue","freq_relative"]]
  for e in fdistcooc_adjMC :
    liste = [e[0],e[1],e[1]/nb_mots_corpus*100]
    dataviz_cooc_adj.append(liste)

  #################################

  fdistcooc_verb = FreqDist(cooc_verb)
  fdistcooc_verbMC = fdistcooc_verb.most_common(10)
  dataviz_cooc_verb = [["terme", "freq_absolue","freq_relative"]]
  for e in fdistcooc_verbMC :
    liste = [e[0],e[1],e[1]/nb_mots_corpus*100]
    dataviz_cooc_verb.append(liste)

  #################################

  fdistcooc_nom = FreqDist(cooc_nom)
  fdistcooc_nomMC = fdistcooc_nom.most_common(10)
  dataviz_cooc_nom = [["terme", "freq_absolue","freq_relative"]]
  for e in fdistcooc_nomMC :
    liste = [e[0],e[1],e[1]/nb_mots_corpus*100]
    dataviz_cooc_nom.append(liste)

                # FILL THE GSHEET

  worksheet_new = sh.add_worksheet(title=perso, rows="100", cols="100")

  sh.values_update(perso+"!A:B", params={'valueInputOption': 'RAW'}, body={'values': dataviz_freq_roman})
  sh.values_update(perso+"!C:D", params={'valueInputOption': 'RAW'}, body={'values': dataviz_freq_annees})
  sh.values_update(perso+"!E:F", params={'valueInputOption': 'RAW'}, body={'values': dataviz_freq_auteur})
  sh.values_update(perso+"!G:H", params={'valueInputOption': 'RAW'}, body={'values': dataviz_freq_editeur})

  sh.values_update(perso+"!I:K", params={'valueInputOption': 'RAW'}, body={'values': dataviz_cooc_global})
  sh.values_update(perso+"!L:N", params={'valueInputOption': 'RAW'}, body={'values': dataviz_cooc_adj})
  sh.values_update(perso+"!O:Q", params={'valueInputOption': 'RAW'}, body={'values': dataviz_cooc_nom})
  sh.values_update(perso+"!R:T", params={'valueInputOption': 'RAW'}, body={'values': dataviz_cooc_verb})

  sh.values_update(perso+"!U:Y", params={'valueInputOption': 'RAW'}, body={'values': dataviz_conco})


# Tableau des fréquences totales

dataviz_freqTotales_lien = [["Terme", "Frequence"]]
dataviz_freqTotales = [["Terme"]]
for terme, freq in freqTotales_personnages.items() :
  lien = "<a href=\"personnage.php?personnage="+terme+"\">"+terme+"</a>"  ####### LIEN DANS TABLEAU CONCORDANCES
  liste_lien = [lien, freq]
  liste_terme = [terme]
  dataviz_freqTotales.append(liste_terme)
  dataviz_freqTotales_lien.append(liste_lien)
sh.values_update("Feuille 1!A:A", params={'valueInputOption': 'RAW'}, body={'values': dataviz_freqTotales})
sh.values_update("Feuille 1!B:C", params={'valueInputOption': 'RAW'}, body={'values': dataviz_freqTotales_lien})

# Tableau nombre de mots par roman
dataviz_nbmotsRoman = [["Roman", "Nombre_de_mots"]]
for roman, nb_mots in dict_nbmots_romans.items():
  liste = [roman, nb_mots]
  dataviz_nbmotsRoman.append(liste)
sh.values_update("Feuille 1!D:E", params={'valueInputOption': 'RAW'}, body={'values': dataviz_nbmotsRoman})

# Nombre de mots du corpus 

dataviz_nb_mots_corpus = [["Nombre_mots_corpus"],[nb_mots_corpus]]
sh.values_update("Feuille 1!F:F", params={'valueInputOption': 'RAW'}, body={'values': dataviz_nb_mots_corpus})


sh.share("None", perm_type='anyone', role='reader')
print("ID du spreadsheet créé et partagé : ", sh.id)


  0%|          | 0/101 [00:00<?, ?it/s]


Calculs lexicométriques sur 101 termes et création d'un fichier Google Sheet contenant les résultats : 


  3%|▎         | 3/101 [00:26<14:18,  8.76s/it]